# Overview

This notebook can be used with both a single or multi- speaker corpus and allows the interactive plotting of speaker embeddings linked to underlying audio (see instructions in the repo's speaker_embedding directory)

Depending on the directory structure used for your corpus, you may need to adjust handling of **speaker_to_utter** and **locations**.

## Imports


In [ ]:
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

import numpy as np
import torch
import umap

from TTS.bin.compute_embeddings import compute_embeddings
from TTS.config import load_config
from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.utils.audio import AudioProcessor

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, BoxZoomTool, ResetTool, OpenURL, TapTool
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10

output_notebook()

For larger sets of speakers, you can use `Category20`, but you need to change it in the `pal` variable too

List of Bokeh palettes here: https://docs.bokeh.org/en/latest/docs/reference/palettes.html

**NB:** if you have problems with other palettes, first see https://stackoverflow.com/questions/48333820/why-do-some-bokeh-palettes-raise-a-valueerror-when-used-in-factor-cmap

## Config

You should adjust all the paths to point at the relevant locations for you locally.

In [ ]:
# Dataset
formatter_name = "ljspeech"
dataset_name = "ljspeech"
dataset_path = "path/to/LJSpeech-1.1"
meta_file_train = "metadata.csv"

# Speaker encoder
se_model_path = "https://github.com/coqui-ai/TTS/releases/download/speaker_encoder_model/model_se.pth.tar"
se_config_path = "https://github.com/coqui-ai/TTS/releases/download/speaker_encoder_model/config_se.json"
embedding_path = "speakers.pth"

In [ ]:
dataset_config = BaseDatasetConfig()
dataset_config.formatter = formatter_name
dataset_config.dataset_name = dataset_name
dataset_config.path = dataset_path
dataset_config.meta_file_train = meta_file_train

meta_data_train, meta_data_eval = load_tts_samples(dataset_config, eval_split=False)
utt_to_wav = {
    item["audio_unique_name"]: str(Path(item["audio_file"]).relative_to(dataset_path)) for item in meta_data_train
}

## Compute embeddings

You can skip this if you have already computed embeddings with `TTS/bin/compute_embeddings.py`

In [ ]:
compute_embeddings(
    model_path=se_model_path,
    config_path=se_config_path,
    output_path=embedding_path,
    formatter_name=formatter_name,
    dataset_name=dataset_name,
    dataset_path=dataset_path,
    meta_file_train=meta_file_train,
)

## Plot Umap

Bring in the embeddings created by `TTS/bin/compute_embeddings.py`

In [ ]:
embeddings = torch.load(embedding_path, weights_only=True)

In [ ]:
speakers = set()
speaker_to_utter = defaultdict(list)
for idx, embedding in embeddings.items():
    speaker = embedding["name"]
    speakers.add(speaker)
    speaker_to_utter[speaker].append(idx)
print(f"Speaker count: {len(speakers)}")

### Set up the embeddings

Adjust the number of speakers to select and the number of utterances from each speaker and they will be randomly sampled from the corpus

In [ ]:
embeds = []
labels = []
locations = []

# single speaker
num_speakers = 1
num_utters = 1000

# multi speaker
# num_speakers = 10
# num_utters = 20

speaker_idxs = random.sample(list(speakers), num_speakers)

for speaker_num, speaker_idx in enumerate(speaker_idxs):
    speakers_utter = speaker_to_utter[speaker_idx]
    utter_idxs = random.sample(speakers_utter, num_utters)
    for utter_idx in utter_idxs:
        embed = np.array(embeddings[utter_idx]["embedding"])
        embeds.append(embed)
        labels.append(speaker_idx)
        locations.append(utt_to_wav[utter_idx])
embeds = np.stack(embeds)

### Load embeddings with UMAP

In [ ]:
model = umap.UMAP()
projection = model.fit_transform(embeds)

### Interactively charting the data in Bokeh

You can use the regular Bokeh [tools](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/tools.html) to explore the data, with reset setting it back to normal

Once you have started the local server (see cell below) you can then click on plotted points which will open a tab to play the audio for that point, enabling easy exploration of your corpus

File location in the tooltip is given relative to **AUDIO_PATH**

In [ ]:
source_wav_stems = ColumnDataSource(
    data=dict(
        x=projection.T[0].tolist(),
        y=projection.T[1].tolist(),
        desc=locations,
        label=labels,
    )
)

hover = HoverTool(tooltips=[("file", "@desc"), ("speaker", "@label")])

### Optionally consider adding these to the tooltips if you want additional detail
# for the coordinates: ("(x,y)", "($x, $y)"),
# for the index of the embedding / wav file: ("index", "$index"),

factors = list(set(labels))
pal_size = max(len(factors), 3)
pal = Category10[pal_size]

p = figure(width=600, height=400, tools=[hover, BoxZoomTool(), ResetTool(), TapTool()])
p.scatter(
    "x",
    "y",
    source=source_wav_stems,
    color=factor_cmap("label", palette=pal, factors=factors),
)

url = "http://localhost:8000/@desc"
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)

show(p)

### Local server to serve wav files from corpus

This is required so that when you click on a data point the hyperlink associated with it will be served the file locally.

There are other ways to serve this if you prefer and you can also run the commands manually on the command line

The server will continue to run until stopped. To stop it simply interupt the kernel (ie square button or under Kernel menu)

In [ ]:
%cd $dataset_path
%pwd
!python -m http.server